# Simple Text Analyzer

ok so here have have some data with some sencetes and tags. We write something and the AI will try to geuss what it's.
 

In [1]:
import json
from io import open
import re
import numpy as np
import pandas as pd

In [2]:
# load data 
with open("data.json", "r") as d:
    data = json.load(d)
data

{'tags': [{'tag': 'greetings',
   'sentences': ['Hello!',
    'Hey',
    "What's up",
    'Nice to meet you',
    'Good morning.',
    'Good evening',
    'Good morning',
    'Good afternoon',
    'Good evening',
    "It's nice to meet you",
    "It's a pleasure to meet you"]},
  {'tag': 'identity',
   'sentences': ['Who are you?',
    'What are you?',
    'What is you name?',
    'What do you do?']},
  {'tag': 'jokes',
   'sentences': ["Did you hear about the mathematician who's afraid of negative numbers?",
    "Why do we tell actors to 'break a leg?'",
    'Helvetica and Times New Roman walk into a bar.',
    'Did you hear about the actor who fell through the floorboards?']}]}

In [3]:
# totalt word list 
wordlist = []
tag_wordlist = dict()
pattern = re.compile('\w+')
for tag in data['tags']:
    tag_wordlist[tag['tag']] = []
    for s in tag['sentences']:
        # print(tag['tag'],[a for a in p.findall(s)])
        wl = [a.lower() for a in pattern.findall(s)]
        tag_wordlist[tag['tag']].extend(wl)
        wordlist.extend(wl)

# wordlist.sort()
wordlist = list(set(wordlist))
wordlist.sort()

In [4]:
# print("tag wl: ",tag_wordlist,'\n')
# print("wl:", wordlist, '\n')
print("len of wordlist:", len(wordlist))
df_tags = pd.DataFrame(enumerate(tag_wordlist))
df_tlk = pd.DataFrame(list(tag_wordlist.keys()), columns=["keys"])
df_tlv = pd.DataFrame(tag_wordlist.values())
df_tlk.join(df_tlv)

len of wordlist: 48


,keys,0,1,2,3,4,5,6,7,8,...,31,32,33,34,35,36,37,38,39,40
0,greetings,hello,hey,what,s,up,nice,to,meet,you,...,you,None,None,None,None,None,None,None,None,None
1,identity,who,are,you,what,are,you,what,is,you,...,None,None,None,None,None,None,None,None,None,None
2,jokes,did,you,hear,about,the,mathematician,who,s,afraid,...,you,hear,about,the,actor,who,fell,through,the,floorboards


So now I'm gonna try to make a matrix of bools/counts,

TagName   | hello | new | why | you    
----------|-------|-----|-----|--------
greetings | 1     | 0   | 1   | 1 or 3 
identity  | 0     | 0   | 0   | 1 or 4 
jokes     | 0     | 1   | 1   | 1 or 2 
 

In [5]:
mat = np.zeros((len(tag_wordlist),len(wordlist)), 'int8')
for iy, word in enumerate(wordlist):
    for ix, tag in enumerate(tag_wordlist):
        for w in tag_wordlist[tag]:
            if w == word:
                mat[ix,iy] += 1


In [6]:
df_mat = pd.DataFrame(mat, columns=wordlist)
df_words_tags = df_tlk.join(df_mat)
df_words_tags

,keys,a,about,actor,actors,afraid,afternoon,and,are,bar,...,through,times,to,up,walk,we,what,who,why,you
0,greetings,1,0,0,0,0,1,0,0,0,...,0,0,3,1,0,0,1,0,0,3
1,identity,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,3,1,0,4
2,jokes,2,2,1,1,1,0,1,0,1,...,1,1,1,0,1,1,0,2,1,2


OK Now we can start modeling...

This is interesting, I do not think I can train this model! 

I don’t have datasets. I just have a collection of words and tags.

I also have notice that I completely, ignore the scenes. But what if …


In [7]:
# lets ckeck and if what tag might 'to' be more likelly to be in 
df_words_tags[["keys", "to"]]

,keys,to
0,greetings,3
1,identity,0
2,jokes,1


In [8]:
# well it's obviuslly 3 
3/(3+1) # is 0.75
# this give me an idea

0.75

In [9]:
# lets try converting all numbers to percentages
def runOnCol(colName: str, df: pd.DataFrame):
    sum = df[colName].sum()
    return df[colName].apply(lambda v: round(v/sum,2))

In [10]:
df_words_tags_p = pd.DataFrame(df_words_tags["keys"], columns=df_words_tags.columns)
for col in df_words_tags.columns[1:]:
    df_words_tags_p[col] = runOnCol(col, df_words_tags)

df_words_tags_p

,keys,a,about,actor,actors,afraid,afternoon,and,are,bar,...,through,times,to,up,walk,we,what,who,why,you
0,greetings,0.33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.75,1.0,0.0,0.0,0.25,0.00,0.0,0.33
1,identity,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.75,0.33,0.0,0.44
2,jokes,0.67,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.25,0.0,1.0,1.0,0.00,0.67,1.0,0.22


In [11]:
# humm I wonder what if I sum it up
tests = [
    "hello",
    "morning",
    "good evening",
    "who are you",
    "what is that",
    "is that moring",
    "two man walked in to a bar",
    "do you have something for me"
]
def testPrediction(val):
    sentence = val.lower().split(' ')
    df= []
    for word in sentence:
        if word in df_words_tags_p:
            df.append(df_words_tags_p[word].to_list())

    return pd.DataFrame(df, columns=df_words_tags_p["keys"]).sum()

In [12]:
for test in tests:
    res = testPrediction(test)
    print(test,'\n', res.sort_values(ascending=False), '\n')


hello 
 keys
greetings    1.0
identity     0.0
jokes        0.0
dtype: float64 

morning 
 keys
greetings    1.0
identity     0.0
jokes        0.0
dtype: float64 

good evening 
 keys
greetings    2.0
identity     0.0
jokes        0.0
dtype: float64 

who are you 
 keys
identity     1.77
jokes        0.89
greetings    0.33
dtype: float64 

what is that 
 keys
identity     1.75
greetings    0.25
jokes        0.00
dtype: float64 

is that moring 
 keys
identity     1.0
greetings    0.0
jokes        0.0
dtype: float64 

two man walked in to a bar 
 keys
jokes        1.92
greetings    1.08
identity     0.00
dtype: float64 

do you have something for me 
 keys
identity     1.11
jokes        0.55
greetings    0.33
dtype: float64 



But this is not machine learning, I'm just running basic statistics. There is no improvement here, that data is constant and stale.

However, this might just be useable for a simple chat bot, if I added word stemming and some responses based on tags. It should work, shouldn't it?

Currently I have ~48 unique words, I believe if I added word stemming that number may be reduced by ~20%, if not more.

Nothing noticeable due to lack of data, but if I reach 5000 words. How accurate can I get?

I have noticed that I can use the test Predictions output to feed back in to the json. Let's say that I take the content of a novel. Break it to sentences, clean up the punctuations and then feed it into the function. And take the results and feed it back, to the tag with the highest value.

Now that might just be Machine learning.
